In [1]:
from collections import Counter

class LexNode:
    def __init__(self, char, kid = None, parent = None, isLastSame = False, isLast = False, isChild = False):
        self.char = char
        self.children = []
        self.property = 0
        self.idx = 0
        self.kid = kid
        self.parent = parent
        self.isLastSame = isLastSame
        self.isLast = isLast
        self.isChild = isChild

    def get_max_level(self, level = 0):
        if len(self.children) == 0:
            return level
        child_levels = [child.get_max_level(level = level + 1) for child in self.children]
        return max(child_levels)

class LexTree:
  def __init__(self, words):
    max_length = max(len(word) for word in words)
    self.words_len = [len(word) for word in words]
    padded_words = [word.ljust(max_length, "0") for word in words]
    self.words = padded_words
    self.root_node = LexNode('*')
    self.root_node.property = 1
    self.word_max_len = max_length

  def convert_to_lex(self):
    word_list_lex = []
    for word in self.words:
        lex_for_each_word = []
        for character in word:
            char = LexNode(character)
            lex_for_each_word.append(char)
        word_list_lex.append(lex_for_each_word)
    return word_list_lex

  def LexTree_build(self):
    word_list_lex = self.convert_to_lex()
    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        if word_list_lex[row][column].char == "0":
          break
        elif row == 0:
          current_node = word_list_lex[row][column]
          if column == self.words_len[row] - 1:
            current_node.isLast = True
        elif column == 0:
          pass
        else:
          previous_node=word_list_lex[row][column - 1]
          current_node=word_list_lex[row][column]
          compare_node=word_list_lex[row - 1][column]
          if current_node.char == compare_node.char:
            continue
          else:
            previous_node.isLastSame = True
            parent_node = word_list_lex[row - 1][column - 1]
            parent_node.kid = previous_node
            previous_node.parent = parent_node
            previous_node.isChild = True
            break

    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        current_node = word_list_lex[row][column]
        if column == self.words_len[row] - 1:
          current_node.isLast = True
    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        if row >= 1 and column >= 1:
          previous_node = word_list_lex[row][column - 1]
          current_node = word_list_lex[row][column]
          parent_node = word_list_lex[row - 1][column]
          if parent_node.isLast and current_node.isLastSame:
            previous_node.isChild = True
            current_node.isChild = False
            parent_node = word_list_lex[row-1][column-1]
            parent_node.kid = previous_node
            previous_node.parent = parent_node
    c = [0]
    for row in range(len(word_list_lex)):
      for column in range(len(word_list_lex[0])):
        current_node=word_list_lex[row][column]
        if current_node.isChild:
          c.append(column)
    d = []
    for row in range(len(word_list_lex)):
      e = []
      for column in range(len(word_list_lex[0])):
        current_node = word_list_lex[row][column]
        e.append(current_node.isLast)
      d.append(e)
    for row in range(len(word_list_lex)):
      str = ""
      for column in range(len(word_list_lex[0])):
        current_node = word_list_lex[row][column]
        if current_node.char == "0":
            break
        elif column == c[row]:
          if row == 0:
            str = str + current_node.char + "--"
          else:
            str = str + "|--"
        elif column < c[row] :
          str = str + "|  "
        elif column > c[row]:
          if not current_node.isLast:
            str = str + current_node.char + "--"
          else:
            str = str + current_node.char
      print(str)
    return word_list_lex,c

  def add_lex_node(self, parent_node, child_node):
        assert isinstance(parent_node, LexNode) and isinstance(child_node, LexNode), "Both parent and child must be LexNode instances"
        parent_node.children.append(child_node)

  def construct_lexicon_tree(self):
        self._recursive_tree_build(self.root_node, self.words_list, 0)

  def _recursive_tree_build(self, current_node, remaining_words, current_index):
        if current_index == self.longest_word_length:
            print("Finished")
        words_ending_here = []
        for word in remaining_words:
            if (current_index + 1 < self.longest_word_length and word[current_index + 1] == '$') or current_index == self.longest_word_length - 1:
                end_node = LexNode(word[current_index])
                end_node.property = 2
                end_node.children.append(self.root_node)
                self.add_lex_node(current_node, end_node)
                words_ending_here.append(word)
        for word in words_ending_here:
            remaining_words.remove(word)
        remaining_word_count = len(remaining_words)
        next_chars = [remaining_words[idx][current_index] for idx in range(remaining_word_count)]
        char_counts = Counter(next_chars)
        for char, count in char_counts.items():
            next_node = LexNode(char)
            self.add_lex_node(current_node, next_node)
            words_for_next_char = [word for word in remaining_words if word[current_index] == char]
            self._recursive_tree_build(next_node, words_for_next_char, current_index + 1)

def find_parent_row(array):
  parents = []
  previous_value = None
  for i, value in enumerate(array):
    if value != previous_value:
      if i == 0:
        parents.append(i)
      else:
        parents.append(i - 1)
      previous_value = value
  return parents

def read_words_from_file(file_path):

    words = []
    try:
        with open(file_path, 'r', encoding = 'utf-8') as file:
            for line in file:
                words_in_line = line.strip().split()
                words.extend(["*" + word for word in words_in_line])
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding = 'windows-1252') as file:
            for line in file:
                words_in_line = line.strip().split()
                words.extend(["*" + word for word in words_in_line])
    return words

file_path = 'dict_1.txt'
word_list = read_words_from_file(file_path)
print(word_list)
Tree = LexTree(word_list)
lex_matrix,c = Tree.LexTree_build()
parents_row = find_parent_row(c)
dict_words = []


['*a', '*abandon', '*abbas', '*abbreviation', '*abdominal', '*abela', '*abernethy', '*abides', '*able', '*abolishing', '*abortionists', '*about', '*abraham', '*abridge', '*absences', '*absolved', '*abstinent', '*abundantly', '*accedes', '*accentuating', '*accept', '*accident', '*accommodated', '*accompany', '*accomplishment', '*accord', '*accountancy', '*accrue', '*accuride', '*acero', '*aches', "*acker's", '*ackroyd', '*acquaint', '*acquit', '*acronym', '*across', '*activate', "*actor's", '*actually', '*acumen', '*adachi', '*adaptec', '*addison', '*adelsberger', '*adirondack', '*adjuncts', '*admirable', '*admits', '*adolf', '*adorabelle', '*adrenaline', '*adult', '*advantage', '*adversaries', '*advice', '*advil', '*advised', '*advocate', '*aeronautical', '*aesthete', '*affairs', '*affirmations', '*affluent', "*afghanistan's", '*africa', '*after', '*aftereffect', '*again', '*against', '*aggregates', '*agility', '*agonizes', '*agreed', '*agreeing', '*agreement', '*agrippa', '*aguilera',

In [2]:
import numpy as np
class LexTreeNode:
    def __init__(self, char):
        self.char = char  #
        self.children = {}  
        self.parent = None
        self.is_end_of_word = False


class LexTree:
    def __init__(self):
        self.root = LexTreeNode("*") 

    def add_word(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = LexTreeNode(char)
            parent_node=node
            node = node.children[char]
            node.parent=parent_node

        node.is_end_of_word = True

    def build_tree(self, words):
        for word in words:
            self.add_word(word)  

    def print_tree(self, node=None, indent="", is_last=True):
        if node is None:
            node = self.root

        
        if node != self.root:
            print(indent + ('|--' if not is_last else '--') + node.char)
            indent += "  " if is_last else "|  "

        if node.children:
            last_child = list(node.children.keys())[-1]
            for child_char in node.children:
                self.print_tree(node.children[child_char], indent, child_char == last_child)


    def search_word(self, word):
        node = self.root
        for char in word[1:]:
            if char in node.children:

                node = node.children[char]
            else:

                return False

        return node.is_end_of_word

    def check_spelling(self, word):
        if self.search_word("*" + word):
            return True, "Word is spelled correctly."
        else:
            return False, "Word might be spelled incorrectly."
    def find_prefix(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []  # Prefix not found
            node = node.children[char]

        return self._find_words_from_node(node, prefix)

    def _find_words_from_node(self, node, prefix):
        words = []
        if node.is_end_of_word:
            words.append(prefix)

        for char, next_node in node.children.items():
            words.extend(self._find_words_from_node(next_node, prefix + char))

        return words
    def find_closest_word(self,word):
        candidates=[]
        prefix=""
        if self.search_word("*"+word):
            return word
        else:
            node=self.root
            for char in word:

                if char in node.children:
                    prefix=prefix+char
                    node=node.children.get(char)
                else:
                    candidates=self.find_prefix(prefix)
                    break
        closest_word = None
        smallest_distance = float('inf')
        for candidate in candidates:
            distance = levenshtein_distance(word, candidate)
            if distance < smallest_distance:
                closest_word = candidate
                smallest_distance = distance

        return closest_word









    def find_suggestions(self, word, beam_width=3):
        candidates = [(self.root, "*", 0)]  
        suggestions = []

        for char in  word: 
            new_candidates = []
            for node, acc_word, _ in candidates:
                for child_char, child_node in node.children.items():
                    new_word = acc_word + child_char
                   
                    score = levenshtein_distance(new_word, '*' + word)
                    new_candidates.append((child_node, new_word, score))

    
            candidates = sorted(new_candidates, key=lambda x: x[2])[:beam_width]

        
        for node, acc_word, score in candidates:
            if node.is_end_of_word:
                suggestions.append((acc_word[1:], score))  

        
        suggestions = sorted(suggestions, key=lambda x: x[1])
        best_suggestions = [suggestion for suggestion, _ in suggestions]

        return best_suggestions[:beam_width]

    def segment_and_spellcheck(self, text, beam_width=3):
        candidates = [(self.root, "", 0, [])]  # (node, accumulated word, score, list of words)
        for char in text:
            new_candidates = []
            for node, acc_word, score, words in candidates:
                node_matched = False
                for child_char, child_node in node.children.items():
                    if child_char == char:
                        new_candidates.append((child_node, acc_word + char, score, words))
                        node_matched = True
                    elif child_char == '*' and node.is_end_of_word:
                        
                        new_candidates.append((self.root, char, score, words + [acc_word]))
                        node_matched = True
                if not node_matched:
                    new_candidates.append((node, acc_word + char, score + 1, words))

          
            candidates = sorted(new_candidates, key=lambda x: x[2])[:beam_width]

        # Attempt to finalize words if the last node was end-of-word
        final_candidates = [(node, acc_word, score, words + [acc_word]) for node, acc_word, score, words in candidates if node.is_end_of_word or acc_word == ""]

        if not final_candidates:  # Fallback if no ideal candidates found
            final_candidates = candidates  # This might need more sophisticated handling

        # Choose the best candidate based on the score; fallback to any candidate if empty
        best_candidate = sorted(final_candidates, key=lambda x: x[2])[0] if final_candidates else candidates[0]
        _, _, _, best_words = best_candidate

        return ' '.join(best_words).strip()


def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]
import math
def find_closest_word(dict, segment):
    min_distance = float('inf')
    closest_word = None
    for word in dict:
        distance = levenshtein_distance(segment, word)
        if distance < min_distance:
            min_distance = distance
            closest_word = word
    return closest_word, min_distance

def segment_text_with_levenshtein(dict, text):
    """
    Segment a string into words using dynamic programming,
    minimizing the cumulative Levenshtein distance.
    """
    n = len(text)
    costs = [float('inf')] * (n + 1)
    costs[0] = 0  # Base case: cost of an empty string is 0
    last_split = [-1] * (n + 1)  # Track the last split position for reconstruction

    for i in range(1, n + 1):
        for j in range(i):
            segment = text[j:i]
            if segment in dict:  # If the segment is a word, no need to correct it
                cost = 0
            else:
                # Find the closest word in lex_tree and calculate the cost
                closest_word, min_distance = find_closest_word(dict, segment)
                cost = min_distance
            new_cost = costs[j] + cost
            if new_cost < costs[i]:
                costs[i] = new_cost
                last_split[i] = j

    # Reconstruct the segmented and corrected text
    corrected_words = []
    i = n
    while i > 0:
        j = last_split[i]
        segment = text[j:i]
        if segment not in dict:
            closest_word, _ = find_closest_word(dict, segment)
            corrected_words.append(closest_word)
            print(closest_word)
        else:
            print(segment)
            corrected_words.append(segment)
        i = j
    corrected_words.reverse()

    return corrected_words



def segment_recursive(text,dict):

    min_distance=float("inf")

    min_idx=0

    for i in range(len(text)):
        minn_distance=float("inf")
        first_word=text[:i+1]
        if first_word not in dict:
            continue
        else:
            for template in dict:
                distance=levenshtein_distance(template,'*'+first_word)
                if distance<minn_distance:
                    minn_distance=distance

        if minn_distance<=min_distance:
            min_distance=minn_distance
            min_idx=i
    first_word=text[:min_idx+1]
    if len(text[min_idx+1:])==0:
        print(first_word)
        return first_word
    else:
        #print(first_word)
        result=first_word+" "+segment_recursive(text[min_idx+1:],dict)

        return result







def load_dictionary(file_path):

    dict_words = []
    with open(file_path, 'r', encoding='latin1') as file:  
        for line in file:
            word = line.strip()  
            if word:  
                dict_words.append(word)
    return dict_words
import string
def load_text1(file_path):

    translator = str.maketrans('', '', string.punctuation)

    words = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:

            cleaned_line = line.translate(translator)
            # Split the line into words and extend the list of words
            words.extend(cleaned_line.split())

    return words
def load_text(file_path):
    """
    从给定路径加载文本文件。
    :param file_path: 文本文件的路径。
    :return: 文本内容的字符串。
    """
    with open(file_path, 'r', encoding='latin1') as file:
        return file.read().strip()  





In [3]:
# Load words from dictionary
dict_words = load_dictionary('dict_1.txt')
print(f"Dictionary loaded with {len(dict_words)} words.")


Dictionary loaded with 6250 words.


In [4]:
lex_tree = LexTree()
lex_tree.build_tree(dict_words)
print("Lexical Tree has been built.")

Lexical Tree has been built.


In [5]:
# Load typos and correct them
typo = 'typos.txt'
test = load_text1(typo)
corrected = ''
for t in test:
    t = '*' + t
    if lex_tree.search_word(t):
        corrected = corrected + t[1:] + " "
    else:
        closest_word = lex_tree.find_closest_word(t[1:])
        corrected = corrected + (closest_word if closest_word else t[1:]) + " "

print("Corrected text:", corrected)


Corrected text: once apostle a tyre wile grammophon was know of benares th bodhisatta kame to lif t the foot of he himalayas as a kon he grew strong and sturdy big of fraim well to do and live by a kerin of th rever bangladeshi in a forest haunt now at that tyer there was a crocodile darlings in th gnarled the kroc maes saw the greate frame of the muncie and she conceived a loaf to etc has harte so she sed to her lord ser i dascenzo to eelpouts the hua of that grating king of the muncie dooms viau sade the crudele i leed in the vari and hee live on dri land hua kan we kackley him dy hua or by crump shee ripe he met be kot if i doanna get heem i shall die all ryser answered th krul kunka hr don't tracie yourself i hav a plan i wil give you his hart to eelpouts so when th bodhisatta wunder sitting on th bank of th gnarled after takin a drink of water the crocodile dromon nyers and seid sir monkey what do yout liv on bad froese in this old family plainer on the odier sycamore of the gange

In [6]:
# File paths for unsegmented text
unsegmented_files = ['unsegmented0.txt', 'unsegmented.txt']
beam_widths = [5, 10, 15]

# Run experiments for each file and beam width
for file_path in unsegmented_files:
    text = load_text(file_path)
    s = segment_recursive(text, dict_words)
    print(f'for the {file_path}, the result is: {s}')



eat
for the unsegmented0.txt, the result is: once upon at i mew h i le brahmadatta was king of benares the bodhisatta came to life at the foot of the himalayas as am on k e y he grew strong and sturdy big off r am ewell to do and lived by a curve of the river ganges in a forest haunt now at that time there was a crocodile dwelling in the ganges the crocodile's mate saw the great frame of them on k e yan d she conceived a longing to eat his hearts o she said to her lord sir i desire to eat the heart of that great king of them on k e y s   
 good wife said the crocodile i live in the water and he lives on dry land how can we catch him 
 by hook or by crook she replied he must be caught if i don't get him is h all die 
 all right answered the crocodile consoling her don't trouble yourself i have a plan i will give you his heart to eat
t
for the unsegmented.txt, the result is: on sea po n at y mew i le g ramada t t a w s k n go f benares t h b oh is at a k am e to l if t the foot of he him